In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [2]:
ls

MadeMeSmile_submissions.zst
Positivity_submissions.zst
UnexpectedlyWholesome_submissions.zst
anxiety_depression_submissions.parquet
anxiety_depression_submissions_preprocessed.parquet
datascience_submissions.zst
depression_submissions.parquet
depression_submissions.zst
depression_submissions_preprocessed.parquet
happy_submissions.parquet
happy_submissions.zst
happy_submissions_preprocessed.parquet
mademesmile_submissions.parquet
mademesmile_submissions_preprocessed.parquet
positivity_submissions.parquet
positivity_submissions_preprocessed.parquet
preprocessed_parquet.zip
raw_parquet.zip
reddit_data_preprocessing_AnxietyDepression.ipynb
reddit_data_preprocessing_Mademesmile.ipynb
reddit_data_preprocessing_Positivity.ipynb
reddit_data_preprocessing_Unexpectedlywholesome.ipynb
reddit_data_preprocessing_datascience.ipynb
reddit_data_preprocessing_depression.ipynb
reddit_data_preprocessing_happy.ipynb
reddit_data_preprocessing_technology.ipynb
reddit_data_preprocessing_v1.ipynb
technology_s

In [ ]:
## step1. load downloaded submission reddit dump 

In [3]:
datascience_submissions =\
    pd.read_json('datascience_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 4000000)

In [4]:
# convert the timestamp to utc
datascience_submissions['created_utc'] =\
    datascience_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [15]:
## step2. download the file into parquet file to speed up data load later stage

In [5]:
datascience_submissions.to_parquet('datascience_submissions.parquet', engine='pyarrow')

In [17]:
## step3. read as parquet file 

In [6]:
datascience_submissions = pd.read_parquet('datascience_submissions.parquet',engine='pyarrow')
datascience_submissions

,downs,link_flair_text,distinguished,media,url,link_flair_css_class,id,edited,num_reports,created_utc,banned_by,name,subreddit,title,author_flair_text,is_self,author,media_embed,permalink,author_flair_css_class,selftext,domain,num_comments,likes,clicked,thumbnail,saved,subreddit_id,ups,approved_by,score,selftext_html,created,hidden,over_18,mod_reports,user_reports,stickied,secure_media,secure_media_embed,report_reasons,gilded,retrieved_on,archived,from,from_id,from_kind,quarantine,hide_score,preview,post_hint,locked,contest_mode,spoiler,suggested_sort,brand_safe,author_cakeday,view_count,thumbnail_height,thumbnail_width,can_gild,is_video,approved_at_utc,banned_at_utc,can_mod_post,is_crosspostable,num_crossposts,parent_whitelist_status,whitelist_status,is_reddit_media_domain,pinned,subreddit_type,crosspost_parent,crosspost_parent_list,mod_note,mod_reason_by,mod_reason_title,no_follow,send_replies,subreddit_subscribers,author_flair_background_color,author_flair_richtext,author_flair_template_id,author_flair_text_color,author_flair_type,category,content_categories,is_original_content,link_flair_richtext,link_flair_text_color,link_flair_type,media_only,post_categories,pwls,removal_reason,rte_mode,visited,wls,link_flair_background_color,link_flair_template_id,previous_visits,media_metadata,author_created_utc,author_fullname,is_meta,subreddit_name_prefixed,author_patreon_flair,gildings,is_robot_indexable,all_awardings,total_awards_received,allow_live_comments,discussion_type,author_premium,awarders,removed_by,removed_by_category,treatment_tags,poll_data,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,gallery_data,is_gallery,call_to_action
0,1.0,None,None,None,http://www.digitaltonto.com/2011/how-social-ne...,None,jcaj9,0,NaN,2011-08-08 10:44:30,NaN,t3_jcaj9,datascience,How Social Network Analysis Solves Real World ...,None,False,mhermans,"{'content': None, 'height': None, 'media_domai...",/r/datascience/comments/jcaj9/how_social_netwo...,None,,digitaltonto.com,0,NaN,0.0,http://thumbs.reddit.com/t3_jcaj9.png,0.0,t5_2sptq,4.0,NaN,3,None,1.312804e+09,0.0,False,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,NaN,NaN,None,None,NaN,None
1,0.0,None,None,None,http://radar.oreilly.com/2010/06/what-is-data-...,None,jp5a3,0,NaN,2011-08-20 21:49:53,NaN,t3_jp5a3,datascience,An in-depth analysis of what data scienceis ? ...,None,False,ch00ey,"{'content': None, 'height': None, 'media_domai...",/r/datascience/comments/jp5a3/an_indepth_analy...,None,,radar.oreilly.com,0,NaN,0.0,http://thumbs.reddit.com/t3_jp5a3.png,0.0,t5_2sptq,3.0,NaN,3,None,1.313881e+09,0.0,False,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,NaN,NaN,None,None,NaN,None
2,0.0,None,None,None,http://bit.ly/rnTY0m,None,jytjm,0,NaN,2011-08-30 06:38:41,NaN,t3_jytjm,datascience,What is Data Science?,None,False,[deleted],"{'content': None, 'height': None, 'media_domai...",/r/datascience/comments/jytjm/what_is_data_sci...,None,,bit.ly,0,NaN,0.0,default,0.0,t5_2sptq,1.0,NaN,1,None,1.314690e+09,0.0,False,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,NaN,None,

In [5]:
## step4. feature selection : select only necessary columns

In [7]:
datascience_submissions =\
    datascience_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
datascience_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,datascience,mhermans,False,None,How Social Network Analysis Solves Real World ...,,http://www.digitaltonto.com/2011/how-social-ne...,2011-08-08 10:44:30
1,datascience,ch00ey,False,None,An in-depth analysis of what data scienceis ? ...,,http://radar.oreilly.com/2010/06/what-is-data-...,2011-08-20 21:49:53
2,datascience,[deleted],False,None,What is Data Science?,,http://bit.ly/rnTY0m,2011-08-30 06:38:41
3,datascience,mhermans,False,None,A programmatic data processing API on top of G...,,https://github.com/maxogden/refine-processor,2011-09-10 13:06:01
4,datascience,mhermans,False,None,A Graph-Based Movie Recommender Engine,,http://markorodriguez.com/2011/09/22/a-graph-b...,2011-09-22 08:12:23
...,...,...,...,...,...,...,...,...
92985,datascience,SquidsAndMartians,False,Projects,Coming up with questions for self-initiated pr...,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 19:39:15
92986,datascience,[deleted],False,Career,Looking for resources,[deleted],,2022-12-31 20:53:32
92987,datascience,Ok_Rabbit_639,False,Projects,Projects to start career as a Data Scientist,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 21:46:08
92988,datascience,hisdudeness_ishere,False,Discussion,landing a DS job without a college degree?,"Hi, what path should someone follow to land a ...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 22:51:55


In [7]:
## step5. basic pre-processing 

In [8]:
# remove blank body posts
datascience_submissions = datascience_submissions[datascience_submissions['selftext'].apply(lambda x : len(x)!=0)]
datascience_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
20,datascience,fourxy,False,None,Ask /r/datascience: Any tips for picking up work?,I am quitting my job and starting a data scien...,http://www.reddit.com/r/datascience/comments/n...,2011-12-16 18:45:35
28,datascience,throwaway_account_2U,False,None,Career Advice,I'm trying to get a job in the data science fi...,http://www.reddit.com/r/datascience/comments/p...,2012-02-15 18:38:03
30,datascience,soAsian,False,None,Which .Net language is better for data mining?,I know C# and VB.net but I heard F# is better ...,http://www.reddit.com/r/datascience/comments/q...,2012-02-22 17:44:17
32,datascience,suitablyvague,False,None,What's the mix of software used in r/datascience?,Getting big into Rapidminer lately and CPLEX/R...,http://www.reddit.com/r/datascience/comments/q...,2012-03-08 23:00:16
34,datascience,suitablyvague,False,None,Google Refine query,So Google Refine is pretty amazing. I've been ...,http://www.reddit.com/r/datascience/comments/q...,2012-03-12 19:37:22
...,...,...,...,...,...,...,...,...
92985,datascience,SquidsAndMartians,False,Projects,Coming up with questions for self-initiated pr...,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 19:39:15
92986,datascience,[deleted],False,Career,Looking for resources,[deleted],,2022-12-31 20:53:32
92987,datascience,Ok_Rabbit_639,False,Projects,Projects to start career as a Data Scientist,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 21:46:08
92988,datascience,hisdudeness_ishere,False,Discussion,landing a DS job without a college degree?,"Hi, what path should someone follow to land a ...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 22:51:55


In [9]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
datascience_submissions[datascience_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
2618,datascience,[deleted],False,None,Does anyone have any practice exercises where ...,[deleted],https://www.reddit.com/r/datascience/comments/...,2015-08-26 20:17:09
2628,datascience,[deleted],False,None,Could Use Some Help With A Work Project?,[deleted],https://www.reddit.com/r/datascience/comments/...,2015-08-27 18:03:35
2629,datascience,[deleted],False,None,The Difference Between Machine Learning and St...,[deleted],http://www.galvanize.com/blog/2015/08/26/why-a...,2015-08-27 19:31:07
2636,datascience,[deleted],False,None,Looking for books and other resources on retai...,[deleted],https://www.reddit.com/r/datascience/comments/...,2015-08-28 15:58:05
2659,datascience,[deleted],False,None,Ashley Madison Hack: analysing the data with t...,[deleted],http://community.wolfram.com/groups/-/m/t/557309,2015-09-01 09:17:48
...,...,...,...,...,...,...,...,...
92983,datascience,patdavidjohnson,False,Career,Data Science / Coding Boot Camps- Worth it or ...,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 19:18:16
92984,datascience,birisix,False,Tooling,Why can't I scraping this site?,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 19:33:38
92985,datascience,SquidsAndMartians,False,Projects,Coming up with questions for self-initiated pr...,[removed],https://www.reddit.com/r/datascience/comments/...,2022-12-31 19:39:15
92986,datascience,[deleted],False,Career,Looking for resources,[deleted],,2022-12-31 20:53:32


In [10]:
datascience_submissions = datascience_submissions[~datascience_submissions['selftext'].isin(opt_out_list)]
print(len(datascience_submissions))
datascience_submissions

25387


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
20,datascience,fourxy,False,None,Ask /r/datascience: Any tips for picking up work?,I am quitting my job and starting a data scien...,http://www.reddit.com/r/datascience/comments/n...,2011-12-16 18:45:35
28,datascience,throwaway_account_2U,False,None,Career Advice,I'm trying to get a job in the data science fi...,http://www.reddit.com/r/datascience/comments/p...,2012-02-15 18:38:03
30,datascience,soAsian,False,None,Which .Net language is better for data mining?,I know C# and VB.net but I heard F# is better ...,http://www.reddit.com/r/datascience/comments/q...,2012-02-22 17:44:17
32,datascience,suitablyvague,False,None,What's the mix of software used in r/datascience?,Getting big into Rapidminer lately and CPLEX/R...,http://www.reddit.com/r/datascience/comments/q...,2012-03-08 23:00:16
34,datascience,suitablyvague,False,None,Google Refine query,So Google Refine is pretty amazing. I've been ...,http://www.reddit.com/r/datascience/comments/q...,2012-03-12 19:37:22
...,...,...,...,...,...,...,...,...
92969,datascience,Tarneks,False,Discussion,What is the difference between an online maste...,I see a lot of comments promoting online maste...,https://www.reddit.com/r/datascience/comments/...,2022-12-31 04:58:34
92977,datascience,Captain___Mutato,False,Discussion,What is the value of model monitoring when the...,Suppose we have a solid ML model lifecycle set...,https://www.reddit.com/r/datascience/comments/...,2022-12-31 14:28:09
92982,datascience,Impossible-Ask4646,False,Career,swe vs ds,"I'm a 29yr old dairy farm manager in Colorado,...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 18:55:53
92988,datascience,hisdudeness_ishere,False,Discussion,landing a DS job without a college degree?,"Hi, what path should someone follow to land a ...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 22:51:55


In [11]:
# step6. final data quality check 

In [11]:
print(f'Author : {datascience_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {datascience_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {datascience_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {datascience_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {datascience_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {datascience_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {datascience_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {datascience_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {datascience_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {datascience_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {datascience_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {datascience_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : fourxy
Subreddit : datascience
Teenager(True/False) : False
Posted : 2011-12-16 18:45:35

Title : Ask /r/datascience: Any tips for picking up work?
Body : I am quitting my job and starting a data science consultancy firm at the first of next year.  Do any data science redditors have tips on how to acquire data science work?  How do you market yourselves?  How do you explain 'data science' to people who need it but aren't familiar with the phrase?  Thanks.



Author : throwaway_account_2U
Subreddit : datascience
Teenager(True/False) : False
Posted : 2012-02-15 18:38:03

Title : Career Advice
Body : I'm trying to get a job in the data science field and failing.

I have an MS in Computer Science and a MA in Statistics plus 2 years of experience as a software developer.  In academia I focused primarily on data analytics, machine learning, regression, ANOVA, and math theory courses such as linear programming, natural language processing, linear algebra, and probability theory.  My 

In [12]:
# step7. save final pre-processed data into parquet
datascience_submissions.to_parquet('datascience_submissions_preprocessed.parquet', engine='pyarrow')
datascience_submissions_preprocessed = pd.read_parquet('datascience_submissions_preprocessed.parquet',engine='pyarrow')
datascience_submissions_preprocessed


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
20,datascience,fourxy,False,None,Ask /r/datascience: Any tips for picking up work?,I am quitting my job and starting a data scien...,http://www.reddit.com/r/datascience/comments/n...,2011-12-16 18:45:35
28,datascience,throwaway_account_2U,False,None,Career Advice,I'm trying to get a job in the data science fi...,http://www.reddit.com/r/datascience/comments/p...,2012-02-15 18:38:03
30,datascience,soAsian,False,None,Which .Net language is better for data mining?,I know C# and VB.net but I heard F# is better ...,http://www.reddit.com/r/datascience/comments/q...,2012-02-22 17:44:17
32,datascience,suitablyvague,False,None,What's the mix of software used in r/datascience?,Getting big into Rapidminer lately and CPLEX/R...,http://www.reddit.com/r/datascience/comments/q...,2012-03-08 23:00:16
34,datascience,suitablyvague,False,None,Google Refine query,So Google Refine is pretty amazing. I've been ...,http://www.reddit.com/r/datascience/comments/q...,2012-03-12 19:37:22
...,...,...,...,...,...,...,...,...
92969,datascience,Tarneks,False,Discussion,What is the difference between an online maste...,I see a lot of comments promoting online maste...,https://www.reddit.com/r/datascience/comments/...,2022-12-31 04:58:34
92977,datascience,Captain___Mutato,False,Discussion,What is the value of model monitoring when the...,Suppose we have a solid ML model lifecycle set...,https://www.reddit.com/r/datascience/comments/...,2022-12-31 14:28:09
92982,datascience,Impossible-Ask4646,False,Career,swe vs ds,"I'm a 29yr old dairy farm manager in Colorado,...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 18:55:53
92988,datascience,hisdudeness_ishere,False,Discussion,landing a DS job without a college degree?,"Hi, what path should someone follow to land a ...",https://www.reddit.com/r/datascience/comments/...,2022-12-31 22:51:55
